<a href="https://colab.research.google.com/github/asad-777/Piaic_Gen-Ai/blob/main/University_Faq_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain google_genai langchain_google_genai chromadb langchain-chroma -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.9 MB/s eta 0:00:0

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings

In [3]:
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')


In [4]:
file_path = "/content/data.csv"

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(file_path)

In [7]:
data = df.apply(lambda row: f"question: {row['Question']}\nAnswer: {row['Answer']}", axis=1).tolist()

In [8]:
full_text = "\n".join(data)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [11]:
chunks = text_splitter.create_documents([full_text])

In [12]:
print(len(chunks))

267


In [13]:
import os
os.environ["GOOGLE_API_KEY"] = api_key

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [15]:
print(len(chunks))

267


In [16]:
import chromadb
import time

client = chromadb.Client()
collection = client.create_collection("university_data_2")

batch_size = 50

for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]
    chunk_embeddings = embeddings.embed_documents([chunk.page_content for chunk in batch])

    collection.add(
        documents=[chunk.page_content for chunk in batch],
        embeddings=chunk_embeddings,
        ids=[str(i + idx) for idx in range(len(batch))],
    )

    time.sleep(10)


In [96]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

In [97]:
from langchain.chains import RetrievalQA
retriever = Chroma(
    client=client,
    collection_name="university_data_2",
    embedding_function=embeddings
).as_retriever()


# Create the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Example usage
query = "What is the application deadline"
result = qa({"query": query})
print(result["result"])

The admission deadline is April 30.

